In [1]:
dts = 'Jzuluaga/atcosim_corpus'
mdl = 'openai/whisper-large-v3'
spl = 'train+test'
print(len(spl.split('+')))
wsp = '-'.join(mdl.split('-')[1:])

print('Dataset: ', dts)
print('Model  : ', mdl)
print('Split  : ', spl)
print('Whisper: ', wsp)

2
Dataset:  Jzuluaga/atcosim_corpus
Model  :  openai/whisper-large-v3
Split  :  train+test
Whisper:  large-v3


In [2]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv() 
token = os.getenv("HF_TOKEN")
login(token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/theresa_lyu/.cache/huggingface/token
Login successful


In [3]:
!pip install datasets
from datasets import load_dataset, Audio
dataset = load_dataset(dts)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
dataset

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'text', 'segment_start_time', 'segment_end_time', 'duration'],
        num_rows: 7638
    })
    test: Dataset({
        features: ['id', 'audio', 'text', 'segment_start_time', 'segment_end_time', 'duration'],
        num_rows: 1901
    })
})

In [4]:
# Select the first four training examples and the first validation example
dataset["train"] = dataset["train"].select(range(400))
dataset["test"] = dataset["test"].select(range(100))

# dataset should now contain 'input_features' and 'labels'
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'audio', 'text', 'segment_start_time', 'segment_end_time', 'duration'],
        num_rows: 400
    })
    test: Dataset({
        features: ['id', 'audio', 'text', 'segment_start_time', 'segment_end_time', 'duration'],
        num_rows: 100
    })
})

In [5]:
!pip install git+https://github.com/openai/whisper.git
!pip install openpyxl

import numpy as np
import pandas as pd
from datetime import datetime
import whisper

df = pd.DataFrame(columns=['split', 'hyp-prmpt', 'hyp-clean', 'ref'])

model = whisper.load_model('-'.join(mdl.split('-')[1:]))

print('Starting inference...')
nato = "alpha,bravo,charlie,delta,echo,foxtrot,golf,hotel,india,juliett,kilo,lima,mike,november,oscar,papa,quebec,romeo,sierra,tango,uniform,victor,whiskey,xray,yankee,zulu"
terminology = "climb, climbing, descend, descending, passing, feet, knots, degrees, direct, maintain, identified, ILS, VFR, IFR, contact, frequency, turn, right, left, heading, altitude, flight, level, cleared, squawk, approach, runway, established, report, affirm, negative, wilco, roger, radio, radar"

for s in spl.split('+'):
    for i in range(len(dataset[s])):
        audio = dataset[s][i]['audio']['array']
        audio = whisper.pad_or_trim(audio)

        if wsp == 'large-v3':
            mel = whisper.log_mel_spectrogram(np.float32(audio), n_mels=128).to(model.device)
        else:
            mel = whisper.log_mel_spectrogram(np.float32(audio)).to(model.device)

        try:
            prompt = 'Air Traffic Control Communications ' + dataset[s][i]['info'].replace('\n', ' ') + ' ' + nato.replace(',',' ') + ' ' + terminology.replace(',',' ')

        except:
            inf = ''
            prompt = 'Air Traffic Control Communications ' + nato.replace(',',' ') + ' ' + terminology.replace(',',' ')

        options = whisper.DecodingOptions(language='en', prompt=prompt, fp16=False)
        res_prmpt = whisper.decode(model, mel, options=options)
        options = whisper.DecodingOptions(language='en', fp16=False)
        res_clean = whisper.decode(model, mel, options=options)

        df.loc[len(df.index)] = [s, res_prmpt.text, res_clean.text, dataset[s][i]['text']]

        print(s, str(int(i/len(dataset[s])*100))+'%', end='\r')
df.to_excel(dts.split('/')[-1]+'-'+spl+'-'+mdl.split('/')[-1]+'-'+datetime.today().strftime('%Y-%m-%d--%H:%M:%S')+'.xlsx')

df

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-unkq70ut
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-unkq70ut
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Starting inference...


,split,hyp-prmpt,hyp-clean,ref
0,train,Bellstar 810 turn right to Drasedingen,BELSTAR 810 turn right to Drasedingen,psa eight one zero turn right to trasadingen
1,train,Lufthansa 5318 contact Zürich 134.6,"Lufthansa 5318, contact Zürich 134.6",lufthansa five three one eight contact zurich ...
2,train,besta 810 contact zurich 133.4,"Belsa 810, contact Zurich 133.4",psa eight one zero contact zurich one three th...
3,train,sabina 481 ryan identified sabina 481 ryan ide...,"Sabina 481, Ryan identified.",sabena four eight one rhein identified
4,train,transit 101 ryan identified at cost traceting in,"Ransweet 101R identified, set course Drasselin...",transwede one zero one rhein identified set co...
...,...,...,...,...
495,test,Speedbird 77 whiskey whiskey right heading 270,"Speedbird 77, Whiskey Whiskey, right heading 270.",speed bird seven seven whiskey whiskey right h...
496,test,Lada 686900 good morning maintain level 350 an...,"Lada 6869, good morning, maintain level 350 an...",lauda six eight six nine good morning maintain...
497,test,Lufthansa 3556 good morning radar contact,"Lufthansa 3556, good morning, radar contact",lufthansa three five five six good morning rad...
498,test,jet aviation 701 descent flight level 320,"Jet Aviation 701, descent flight level 320.",jet aviation seven zero one descend to flight ...


In [6]:
# Normalization
from Normalizer import filterAndNormalize

df['ref-norm'] = df.apply(lambda x: filterAndNormalize(x['ref']), axis=1)
df['hyp-clean-norm'] = df.apply(lambda x: filterAndNormalize(x['hyp-clean']), axis=1)
df['hyp-prmpt-norm'] = df.apply(lambda x: filterAndNormalize(x['hyp-prmpt']), axis=1)

df.head()

,split,hyp-prmpt,hyp-clean,ref,ref-norm,hyp-clean-norm,hyp-prmpt-norm
0,train,Bellstar 810 turn right to Drasedingen,BELSTAR 810 turn right to Drasedingen,psa eight one zero turn right to trasadingen,psa 8 1 0 turn right to trasadingen,belstar 8 1 0 turn right to drasedingen,bellstar 8 1 0 turn right to drasedingen
1,train,Lufthansa 5318 contact Zürich 134.6,"Lufthansa 5318, contact Zürich 134.6",lufthansa five three one eight contact zurich ...,lufthansa 5 3 1 8 contact zurich 1 3 4 6,lufthansa 5 3 1 8 contact zrich 1 3 4 6,lufthansa 5 3 1 8 contact zrich 1 3 4 6
2,train,besta 810 contact zurich 133.4,"Belsa 810, contact Zurich 133.4",psa eight one zero contact zurich one three th...,psa 8 1 0 contact zurich 1 3 3 4,belsa 8 1 0 contact zurich 1 3 3 4,besta 8 1 0 contact zurich 1 3 3 4
3,train,sabina 481 ryan identified sabina 481 ryan ide...,"Sabina 481, Ryan identified.",sabena four eight one rhein identified,sabena 4 8 1 rhein identified,sabina 4 8 1 ryan identified,sabina 4 8 1 ryan identified sabina 4 8 1 ryan...
4,train,transit 101 ryan identified at cost traceting in,"Ransweet 101R identified, set course Drasselin...",transwede one zero one rhein identified set co...,transwede 1 0 1 rhein identified set course tr...,ransweet 1 0 1 romeo identified set course dra...,transit 1 0 1 ryan identified at cost tracetin...


In [7]:
!pip install jiwer
import jiwer

def calcWER(df, spl):
    dff = df.loc[df['split'].isin(spl.split('+'))]
    wer_cln = jiwer.wer(list(dff['ref']), list(dff['hyp-clean']))
    wer_prm = jiwer.wer(list(dff['ref']), list(dff['hyp-prmpt']))
    wer_cln_nrm = jiwer.wer(list(dff['ref-norm']), list(dff['hyp-clean-norm']))
    wer_prm_nrm = jiwer.wer(list(dff['ref-norm']), list(dff['hyp-prmpt-norm']))

    print('clean        : {} %'.format(round(wer_cln*100,4)))
    print('prmpt        : {} %'.format(round(wer_prm*100,4)))
    print('clean-norm   : {} %'.format(round(wer_cln_nrm*100,4)))
    print('prmpt-norm   : {} %'.format(round(wer_prm_nrm*100,4)))

# Split Train+Test
spl = 'train+test'
wsp = '-'.join(mdl.split('-')[1:])

print('Dataset: ', dts)
print('Model  : ', mdl)
print('Split  : ', spl)
print('Whisper: ', wsp)

calcWER(df, spl)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Dataset:  Jzuluaga/atcosim_corpus
Model  :  openai/whisper-large-v3
Split  :  train+test
Whisper:  large-v3
clean        : 80.4236 %
prmpt        : 84.2778 %
clean-norm   : 14.7413 %
prmpt-norm   : 30.7079 %


In [8]:
# Split test
spl = 'test'
wsp = '-'.join(mdl.split('-')[1:])

print('Dataset: ', dts)
print('Model  : ', mdl)
print('Split  : ', spl)
print('Whisper: ', wsp)

calcWER(df, spl)

Dataset:  Jzuluaga/atcosim_corpus
Model  :  openai/whisper-large-v3
Split  :  test
Whisper:  large-v3
clean        : 82.3694 %
prmpt        : 77.5187 %
clean-norm   : 12.5468 %
prmpt-norm   : 24.7191 %
